 ### 4-5. SentencePiece 하이퍼파라미터 튜닝 및 성능 개선 실험

#### 데이터 & SentencePiece 준비

In [6]:
import pandas as pd
import os

train_file = os.getenv('HOME') + '/aiffel/sp_tokenizer/naver_data/ratings_train.txt'
test_file = os.getenv('HOME') + '/aiffel/sp_tokenizer/naver_data/ratings_test.txt'

# 데이터 불러오기
train_data = pd.read_csv(train_file, sep='\t')
test_data = pd.read_csv(test_file, sep='\t')

# 데이터 확인
print(train_data.head())
print(test_data.head())


         id                                           document  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1
        id                                           document  label
0  6270596                                                굳 ㅋ      1
1  9274899                               GDNTOPCLASSINTHECLUB      0
2  8544678             뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아      0
3  6825595                   지루하지는 않은데 완전 막장임... 돈주고 보기에는....      0
4  6723715  3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??      0


In [7]:
# 중복 제거
train_data.drop_duplicates(subset=['document'], inplace=True)
test_data.drop_duplicates(subset=['document'], inplace=True)

# 특수 문자 제거 (정규 표현식 사용)
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", regex=True)

print(f"전처리 후 학습 데이터 개수: {len(train_data)}")
print(f"전처리 후 테스트 데이터 개수: {len(test_data)}")

전처리 후 학습 데이터 개수: 146183
전처리 후 테스트 데이터 개수: 49158


In [8]:
# 결측값(NaN) 제거
train_data = train_data.dropna(subset=['document']).copy()
test_data = test_data.dropna(subset=['document']).copy()

# 문장 길이 필터링 (NaN 방지)
filtered_train_data = train_data[train_data['document'].apply(lambda x: 1 <= len(str(x)) <= 140)].copy()
filtered_test_data = test_data[test_data['document'].apply(lambda x: 1 <= len(str(x)) <= 140)].copy()

print(f"필터링 후 학습 데이터 개수: {len(filtered_train_data)}")
print(f"필터링 후 테스트 데이터 개수: {len(filtered_test_data)}")


필터링 후 학습 데이터 개수: 145791
필터링 후 테스트 데이터 개수: 48995


In [1]:
!head -n 5 $HOME/aiffel/sp_tokenizer/data/nsmc_corpus.txt


아 더빙 진짜 짜증나네요 목소리
흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나
너무재밓었다그래서보는것을추천한다
교도소 이야기구먼 솔직히 재미는 없다평점 조정
사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다


In [2]:
!ls -lh $HOME/aiffel/sp_tokenizer/data/nsmc_spm*

-rw-r--r-- 1 root root 369K Feb 25 05:42 /aiffel/aiffel/sp_tokenizer/data/nsmc_spm.model
-rw-r--r-- 1 root root 144K Feb 25 05:42 /aiffel/aiffel/sp_tokenizer/data/nsmc_spm.vocab


#### SentencePiece 모델 로드

In [3]:
import sentencepiece as spm
import os

# 학습된 SentencePiece 모델 로드
model_path = os.getenv('HOME') + '/aiffel/sp_tokenizer/data/nsmc_spm.model'
s = spm.SentencePieceProcessor()
s.Load(model_path)

# 테스트 문장
test_sentence = "이 영화 정말 재미있다!"
print("토큰 ID:", s.EncodeAsIds(test_sentence))
print("토큰화된 문장:", s.EncodeAsPieces(test_sentence))


토큰 ID: [19, 5, 21, 1144, 0]
토큰화된 문장: ['▁이', '▁영화', '▁정말', '▁재미있다', '!']


#### sp_tokenize() 함수 구현

In [4]:
import tensorflow as tf

def sp_tokenize(s, corpus): 
    tensor = []

    # 문장을 토큰 ID 리스트로 변환
    for sen in corpus:
        tensor.append(s.EncodeAsIds(sen))

    # 단어-인덱스 매핑 생성
    vocab_file = os.getenv('HOME') + '/aiffel/sp_tokenizer/data/nsmc_spm.vocab'

    with open(vocab_file, 'r', encoding='utf-8') as f:
        vocab = f.readlines()

    word_index = {}
    index_word = {}

    for idx, line in enumerate(vocab):
        word = line.split("\t")[0]  # 단어 추출

        word_index.update({word: idx})  # 단어 -> 인덱스 매핑
        index_word.update({idx: word})  # 인덱스 -> 단어 매핑

    # 패딩 적용
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

    return tensor, word_index, index_word


#### sp_tokenize() 함수 테스트

In [5]:
# 테스트 데이터 준비
sample_sentences = ["이 영화 정말 재미있어요!", "완전 최악이야.", "기대 이상이었습니다."]

# 토큰화 실행
tokenized_tensor, word_index, index_word = sp_tokenize(s, sample_sentences)

# 결과 확인
print("Tokenized Tensor:", tokenized_tensor)
print("Word Index 예시:", list(word_index.items())[:10])  # 일부 단어 출력
print("Index Word 예시:", list(index_word.items())[:10])  # 일부 단어 출력


Tokenized Tensor: [[  19    5   21 1366    0]
 [ 116  383  871    0    0]
 [ 215  431 4729    0    0]]
Word Index 예시: [('<unk>', 0), ('<s>', 1), ('</s>', 2), ('▁', 3), ('이', 4), ('▁영화', 5), ('의', 6), ('도', 7), ('가', 8), ('는', 9)]
Index Word 예시: [(0, '<unk>'), (1, '<s>'), (2, '</s>'), (3, '▁'), (4, '이'), (5, '▁영화'), (6, '의'), (7, '도'), (8, '가'), (9, '는')]


#### 네이버 영화 리뷰 전체 데이터셋 변환

In [9]:
# 훈련 및 테스트 데이터 변환
train_tensor, train_word_index, train_index_word = sp_tokenize(s, filtered_train_data['document'].tolist())
test_tensor, test_word_index, test_index_word = sp_tokenize(s, filtered_test_data['document'].tolist())

print("변환된 훈련 데이터 크기:", train_tensor.shape)
print("변환된 테스트 데이터 크기:", test_tensor.shape)


변환된 훈련 데이터 크기: (145791, 116)
변환된 테스트 데이터 크기: (48995, 107)


#### 감성 분석 모델 설계
##### Org

In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

# 하이퍼파라미터 설정
vocab_size = 8000  # SentencePiece에서 설정한 vocab_size와 동일
embedding_dim = 128
hidden_units = 64 # 감정 분석 위해

# 모델 설계
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True),
    LSTM(hidden_units),
    Dense(1, activation='sigmoid')  # 감정 분석 (이진 분류)
])

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 구조 확인
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1024000   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 1,073,473
Trainable params: 1,073,473
Non-trainable params: 0
_________________________________________________________________


#### embedding_dim 변경 실험 (256, 512)

In [ ]:
for embedding_dim in [256, 512]:
    print(f"📌 실험: embedding_dim = {embedding_dim}")

    model = Sequential([
        Embedding(input_dim=8000, output_dim=embedding_dim, mask_zero=True),
        LSTM(64),
        Dense(1, activation='sigmoid')
    ])

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # 레이블 데이터 준비
    train_labels = filtered_train_data['label'].values
    test_labels = filtered_test_data['label'].values

    # 모델 학습
    model.fit(train_tensor, train_labels, validation_data=(test_tensor, test_labels),
              epochs=5, batch_size=64, verbose=2)

    # 테스트 평가
    test_loss, test_acc = model.evaluate(test_tensor, test_labels)
    print(f"📊 embedding_dim = {embedding_dim}, 테스트 정확도: {test_acc:.4f}")


📌 실험: embedding_dim = 256
Epoch 1/5
2278/2278 - 673s - loss: 0.3769 - accuracy: 0.8281 - val_loss: 0.3384 - val_accuracy: 0.8497
Epoch 2/5


### 회고

생각보다 전체적으로 학습 시간이 다소 소요됐다.   
그래도 KoNLPy 비교 실험 까지는 수행했는데 Okt가 생각보다 너무 오래 걸려서 실험에서 제외했다.   
또한 모델 파라미터 변경 실험도 굉장히 느리게 진행 되고 있다.   
사실 시간이 좀 더 여유가 있었으면 vocab_size 변경 실험은 꼭 해보고 싶었는데 추후 여유가 있을 때 수행하고 싶다.  
운이 좋은 건지 (?) 첫 시도에 바로 SentencePiece RNN 모델 테스트에서 바로 결과가 0.8이 넘었는데 정확도를 더 높일 수 있는 방법도 좀 생각해봐야겠다.   